In [42]:
# imports and stuff
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

The Data I chose: Spotify Music Dataset from kaggle ( https://www.kaggle.com/datasets/solomonameh/spotify-music-dataset )
It consits of two csv files; high popularity and low popularity

In [43]:
# Importing the data
high = pd.read_csv('../data/high_popularity_spotify_data.csv')
low = pd.read_csv('../data/low_popularity_spotify_data.csv')
# Could I create a merged dataset with an extra 'popularity' column to distinguish the two?

In [44]:
print('high', high.columns)
# print('high', low.columns)

print('high', high.shape)
print('low', high.shape)

high Index(['energy', 'tempo', 'danceability', 'playlist_genre', 'loudness',
       'liveness', 'valence', 'track_artist', 'time_signature', 'speechiness',
       'track_popularity', 'track_href', 'uri', 'track_album_name',
       'playlist_name', 'analysis_url', 'track_id', 'track_name',
       'track_album_release_date', 'instrumentalness', 'track_album_id',
       'mode', 'key', 'duration_ms', 'acousticness', 'id', 'playlist_subgenre',
       'type', 'playlist_id'],
      dtype='object')
high (1686, 29)
low (1686, 29)


Both have the same columns (and even the same amount of samples)
As they already have a 'track_popularity' column, I might not actually need to create one to distinguish the two,
if I know the threshold for high & low priority.\
Of course, the question arises, how chose this threshold and why. Maybe looking into the domain of this column can tell me more.\
I think a first interesting research question could be:"What parameters influence (the most), whether a song is popular or not

In [45]:
print('high max popularity level', high['track_popularity'].max())
print('high min popularity level', high['track_popularity'].min())
print('low max popularity level', low['track_popularity'].max())
print('low min popularity level' , low['track_popularity'].min())

high max popularity level 100
high min popularity level 68
low max popularity level 68
low min popularity level 11


Looks like the domain is 0-100, i.e. a percentage value and the threshold for low/high popularity seems to be 68.
Maybe, as both datasets have the same amount of observations, it's just the median of the whole dataset\
Let's look at the 5 most popular and unpopular songs of the datasets

In [46]:
high = high.sort_values(by='track_popularity', ascending=False)
print("Most popular:\n", high[["track_name", "track_id", "track_artist", "track_album_name", "track_popularity"]].head(5))

Most popular:
            track_name                track_id           track_artist  \
676  Die With A Smile  2plbrEY59IikOBgBGLjaoe  Lady Gaga, Bruno Mars   
455  Die With A Smile  2plbrEY59IikOBgBGLjaoe  Lady Gaga, Bruno Mars   
0    Die With A Smile  2plbrEY59IikOBgBGLjaoe  Lady Gaga, Bruno Mars   
677              APT.  5vNRhkKd0yEAg8suGBpjeY       ROSÉ, Bruno Mars   
4                APT.  5vNRhkKd0yEAg8suGBpjeY       ROSÉ, Bruno Mars   

     track_album_name  track_popularity  
676  Die With A Smile               100  
455  Die With A Smile               100  
0    Die With A Smile               100  
677              APT.                98  
4                APT.                98  


No Taylor Swift, what a nice surprise. I'm guessing this data is from when "Die with a smile" just came out.
I already see some problems with the dataset: Songs are saved multiple times (not sure if they are different versions, or just the same versions e.g. as single and as part of an album) However, as title is saved here, it should be no problem removing duplicates.

In [47]:
display(high[:5])

,energy,tempo,danceability,playlist_genre,loudness,liveness,valence,track_artist,time_signature,speechiness,...,instrumentalness,track_album_id,mode,key,duration_ms,acousticness,id,playlist_subgenre,type,playlist_id
676,0.592,157.969,0.521,pop,-7.777,0.122,0.535,"Lady Gaga, Bruno Mars",3,0.0304,...,0.0,10FLjwfpbxLmW8c25Xyc2N,0,6,251668,0.3080,2plbrEY59IikOBgBGLjaoe,global,audio_features,37i9dQZEVXbMDoHDwVN2tF
455,0.592,157.969,0.521,gaming,-7.777,0.122,0.535,"Lady Gaga, Bruno Mars",3,0.0304,...,0.0,10FLjwfpbxLmW8c25Xyc2N,0,6,251668,0.3080,2plbrEY59IikOBgBGLjaoe,modern,audio_features,37i9dQZF1DWTyiBJ6yEqeu
0,0.592,157.969,0.521,pop,-7.777,0.122,0.535,"Lady Gaga, Bruno Mars",3,0.0304,...,0.0,10FLjwfpbxLmW8c25Xyc2N,0,6,251668,0.3080,2plbrEY59IikOBgBGLjaoe,mainstream,audio_features,37i9dQZF1DXcBWIGoYBM5M
677,0.783,149.027,0.777,pop,-4.477,0.355,0.939,"ROSÉ, Bruno Mars",4,0.2600,...,0.0,2IYQwwgxgOIn7t3iF6ufFD,0,0,169917,0.0283,5vNRhkKd0yEAg8suGBpjeY,global,audio_features,37i9dQZEVXbMDoHDwVN2tF
4,0.783,149.027,0.777,pop,-4.477,0.355,0.939,"ROSÉ, Bruno Mars",4,0.2600,...,0.0,2IYQwwgxgOIn7t3iF6ufFD,0,0,169917,0.0283,5vNRhkKd0yEAg8suGBpjeY,mainstream,audio_features,37i9dQZF1DXcBWIGoYBM5M


In [48]:
low = low.sort_values(by='track_popularity', ascending=False)
print("Most unpopular:\n", low[["track_name", "track_id", "track_artist", "track_album_name", "track_popularity"]].tail(5))

Most unpopular:
                   track_name                track_id   track_artist  \
3009          Hit The Ground  1we3IyLdDkmtZctE9xbFgm  Lilly Ahlberg   
2274      A Vaibele A Tsnien  03bEE0edmcuf9iraKHlENO    Dave Tarras   
3081                 Make It  19bfMEAOJGHEAL2pS3gNoS  Daniel Schulz   
3128  Blue Flag / Fun Things  6VvRwqQpgWU1WclXgCOKI8  Jesper Jenset   
3102            It Ain't You  4egXSi9VmYuJaq6Xa1ECQm      Hedda Mae   

                                track_album_name  track_popularity  
3009                              Hit The Ground                11  
2274  Yiddish-American Klezmer Music - 1925-1956                11  
3081                                     Make It                11  
3128                      Blue Flag / Fun Things                11  
3102                         This Might Get Loud                11  


## Cleaning and combining the 2 datasets
As I know the "Threshold" by which a song is classified as high/low popular I can just merge the two datasets. (however I still wonder how this values came to be)\
I also want to remove duplicates. Above I can see that in this case the 3 duplicates of "Die With A Smile" actually have the same track_id.
I do however wonder, if there may also be duplicates with just the same names (i.e. different versions of the song)\
I'm not quite sure yet, how I want to handle that. But first I want to test if there are any such cases anyway

1. Combining the datasets

In [49]:
print(high.columns, "\n", low.columns) # Looking manually they seem to have the same columns, but for some reason in different order
print(sorted(high.columns) == sorted(low.columns)) # Sorting them to compare their column names elementwise

Index(['energy', 'tempo', 'danceability', 'playlist_genre', 'loudness',
       'liveness', 'valence', 'track_artist', 'time_signature', 'speechiness',
       'track_popularity', 'track_href', 'uri', 'track_album_name',
       'playlist_name', 'analysis_url', 'track_id', 'track_name',
       'track_album_release_date', 'instrumentalness', 'track_album_id',
       'mode', 'key', 'duration_ms', 'acousticness', 'id', 'playlist_subgenre',
       'type', 'playlist_id'],
      dtype='object') 
 Index(['time_signature', 'track_popularity', 'speechiness', 'danceability',
       'playlist_name', 'track_artist', 'duration_ms', 'energy',
       'playlist_genre', 'playlist_subgenre', 'track_href', 'track_name',
       'mode', 'uri', 'type', 'track_album_release_date', 'analysis_url', 'id',
       'instrumentalness', 'track_album_id', 'playlist_id', 'track_id',
       'valence', 'key', 'tempo', 'loudness', 'acousticness', 'liveness',
       'track_album_name'],
      dtype='object')
True


As their equality check returned true, we can just go ahead and combine the two

In [65]:
print(type(low), type(high))
songs = pd.concat([low, high])
display(songs)

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


,time_signature,track_popularity,speechiness,danceability,playlist_name,track_artist,duration_ms,energy,playlist_genre,playlist_subgenre,...,track_album_id,playlist_id,track_id,valence,key,tempo,loudness,acousticness,liveness,track_album_name
2190,4.0,68,0.2330,0.711,Desi Hip-Hop,"Shubh, Ikky",148188.0,0.70200,hip-hop,desi,...,44aCFqUbG06fL58kqpMp8p,37i9dQZF1DX2RahGIyQXcJ,42TMa2hgBNjte4uV7jNCnQ,0.674,5.0,180.219,-10.048,0.00487,0.0932,Baller
31,3.0,68,0.0432,0.184,Classical Essentials,"Ludwig van Beethoven, Paul Lewis",315427.0,0.00527,classical,essential,...,4bNwPPpk01D8pVV9IFSBde,37i9dQZF1DWWEJlAGA9gs0,3DNRdudZ2SstnDCVKFdXxG,0.151,1.0,170.612,-37.264,0.99500,0.1730,"Beethoven: Piano Sonatas, Vol.3"
2189,4.0,68,0.3630,0.656,Desi Hip-Hop,Dhanda Nyoliwala,175909.0,0.60600,hip-hop,desi,...,2HKS1DAJvHmsYs2ORrMQE1,37i9dQZF1DX2RahGIyQXcJ,5nIrIdLPPXeRMl8WXnnDv1,0.380,9.0,175.917,-5.948,0.33500,0.1050,DNW Vol. 1
831,4.0,68,0.0280,0.651,Classic Blues,Michael Jackson,345600.0,0.40200,blues,classic,...,3OBhnTLrvkoEEETjFA3Qfk,5arFyak5nEhUf9kpsssXxT,3AoeaZs8dFemFJr3JdzOL0,0.258,11.0,119.873,-9.303,0.63900,0.0765,"HIStory - PAST, PRESENT AND FUTURE - BOOK I"
2405,5.0,68,0.0646,0.564,Vaporwave Essentials,Home,212939.0,0.75400,electronic,vaporwave,...,1GI4xblPLj1oISUaQhj26P,37i9dQZF1EIdOtkojz5FWw,4E1Zf6j2VSoRPiVBnUCjat,0.457,8.0,170.084,-8.849,0.04240,0.2240,Odyssey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188,3.0,68,0.0263,0.636,Classic Blues,Whitney Houston,238467.0,0.26400,blues,classic,...,2MH37enG6IPvNK5QFLyKes,5arFyak5nEhUf9kpsssXxT,1lx8ddGT5wCD6W2xmLeRKG,0.264,6.0,99.304,-14.567,0.59300,0.2750,Whitney Houston
1607,4.0,68,0.0425,0.741,Afro House,"James Mac, Vall, Rosalie",201713.0,0.87900,electronic,afro house,...,3vmR5tggRYJhVhuB0j1uVc,487jKTFqWhs6b0AEUz0WpX,1aRd5tMVhiRyMJLmSOg2IF,0.362,1.0,123.004,-6.132,0.08410,0.1260,The Boy Is Mine
1676,4.0,68,0.0626,0.792,Afro-Latin Party,Ayra Starr,185093.0,0.50300,latin,afro-latin,...,6CvEsGBD3JdbDKpmJaXn2E,0oU30cCr8klmMsuOKHDLkh,1rrqJ9QkOBYJlsZgqqwxgB,0.381,1.0,99.970,-8.044,0.03690,0.0959,Rush
1665,4.0,68,0.0384,0.753,French Touch,Werenoi,146441.0,0.71700,electronic,french,...,6p4UuRtrBi7EzUVYJdMWDJ,58zteW83Lvl8e158fCCFVN,3kzXT2poHInDISrXOgT1PR,0.601,1.0,117.952,-8.438,0.15100,0.0988,Telegram
